In [3]:
import json
from neo4j import GraphDatabase
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.llms import Ollama
import os
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from neo4j import GraphDatabase 
import re
import ast
import openai
from neo4j import GraphDatabase
from langchain_openai import ChatOpenAI


ModuleNotFoundError: No module named 'neo4j'

In [ ]:
uri = os.environ["NEO4J_URI"]
user=os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]
OPENAI_API_KEY = os.getenv["OPENAI_API_KEY"] 

driver = GraphDatabase.driver(uri, auth=(user, password))
llm = ChatOpenAI(temperature=0, model_name="gpt-4o", api_key=OPENAI_API_KEY)

In [ ]:
# Initialize the Ollama model
ollama = Ollama(base_url='http://localhost:11434', model="llama3.1:70b")

/tmp/ipykernel_3034644/2927552202.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama = Ollama(base_url='http://localhost:11434', model="llama3.1:70b")


In [ ]:
# Utility: Convert string to list
def str_to_list(answer):
    match = re.search(r"\[.*\]", answer)
    try:
        if match:
            match2 = match.group(0)
            pattern = r"(?<=\[)(.*?)(?=\])"
            ans = re.sub(pattern, "", match2)
            answer_list = ast.literal_eval(ans)
        else:
            answer_list = []
    except:
        answer_list = []
    return answer_list

In [ ]:
client = openai.OpenAI(api_key=OPENAI_API_KEY)

def ask_openai(question, model="gpt-4o"):
    """Sends a question to OpenAI's API and returns the response."""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": question}]
    )
    return response.choices[0].message.content

In [ ]:
def query_llm_with_context(ollama, query, matching_results):
    if not matching_results:
        return "@@@@@@@@@@@@I do not know."

    context = []
    for result in matching_results:
        node_a = result["NodeA"]
        node_b = result["NodeB"]
        relationship = result["Relationship"]

        context_entry = f"Node A: {node_a.get('Values', [])}"

        # Get documentId from nodeA if available
        if node_a.get("Properties") and node_a.get("Values"):
            node_a_doc_id = next(
                (v for k, v in zip(node_a["Properties"], node_a["Values"]) if k == "documentId"), None
            )
            if node_a_doc_id:
                context_entry += f" | Node A DocumentId: {node_a_doc_id}"

        if relationship and relationship.get("Type"):
            context_entry += f" | Relationship: {relationship['Type']}"
            context_entry += f" | Relationship Properties: {relationship.get('Properties')}"
            context_entry += f" | Relationship Values: {relationship.get('Values')}"

            if relationship.get("Properties") and relationship.get("Values"):
                rel_doc_id = next(
                    (v for k, v in zip(relationship["Properties"], relationship["Values"]) if k == "documentId"),
                    None
                )
                if rel_doc_id:
                    context_entry += f" | Relationship DocumentId: {rel_doc_id}"

        if node_b.get("ID"):
            context_entry += f" | Node B: {node_b.get('Values', [])}"

            if node_b.get("Properties") and node_b.get("Values"):
                node_b_doc_id = next(
                    (v for k, v in zip(node_b["Properties"], node_b["Values"]) if k == "documentId"), None
                )
                if node_b_doc_id:
                    context_entry += f" | Node B DocumentId: {node_b_doc_id}"

        context.append(context_entry)

    formatted_context = "\n".join(context)

    prompt = f"""
    Context:
    {formatted_context}

    Query: {query}

    Answer only using the information provided in the context above. 
    If the answer is not present in the context, respond with "Unknown"
    Do not make up or guess any information.
    """

    try:
        response = ask_openai(prompt)
        return response
    except Exception as e:
        return f"Error querying LLM: {str(e)}"


In [ ]:
def match_keywords_with_relevance(driver, keywords, database):
    """
    Matches extracted keywords with nodes, properties, and relationships in the Neo4j graph.
    Calculates match percentages and displays top matches.

    :param driver: Neo4j driver connection.
    :param keywords: List of keywords to search for.
    :param database: Name of the Neo4j database.
    :return: List of top matches with match percentages.
    """
    query = """
    OPTIONAL MATCH (n)-[r]-(m)
    WHERE ANY(keyword IN $keywords WHERE 
        ANY(prop IN keys(n) WHERE toLower(toString(n[prop])) CONTAINS toLower(keyword)) OR
        ANY(label IN labels(n) WHERE toLower(label) CONTAINS toLower(keyword)) OR
        ANY(prop IN keys(r) WHERE toLower(toString(r[prop])) CONTAINS toLower(keyword)) OR
        ANY(label IN labels(m) WHERE toLower(label) CONTAINS toLower(keyword)) OR
        ANY(prop IN keys(m) WHERE toLower(toString(m[prop])) CONTAINS toLower(keyword))
    )
    RETURN DISTINCT 
        n AS NodeA,
        type(r) AS RelationshipType,
        keys(r) AS RelationshipProperties,
        [prop IN keys(r) | r[prop]] AS RelationshipValues,
        m AS NodeB,
        keys(n) AS NodeAProperties, 
        [prop IN keys(n) | n[prop]] AS NodeAValues,
        keys(m) AS NodeBProperties, 
        [prop IN keys(m) | m[prop]] AS NodeBValues
    LIMIT 100
    """
    with driver.session(database=database) as session:
        result = session.run(query, keywords=keywords)
        matches = []
        for record in result:
            node_a = record["NodeA"]
            node_b = record["NodeB"]
            relationship_type = record["RelationshipType"]
            relationship_props = record["RelationshipProperties"]
            relationship_values = record["RelationshipValues"]

            # Collect all properties and values for matching
            node_a_props = record["NodeAValues"] if node_a else []
            node_b_props = record["NodeBValues"] if node_b else []

            all_values = (
                (node_a_props if node_a_props is not None else []) +
                (node_b_props if node_b_props is not None else []) +
                (relationship_values if relationship_values is not None else [])
            )


            # Calculate match percentage
            matched_keywords = [
                keyword for keyword in keywords
                if any(keyword.lower() in str(value).lower() for value in all_values)
            ]
            # Calculate match percentage safely
            if len(keywords) > 0:
                match_percentage = (len(matched_keywords) / len(keywords)) * 100
            else:
                match_percentage = 0  # If no keywords are provided, set match percentage to 0


            matches.append({
                "NodeA": {
                    "ID": node_a.element_id if node_a else None,
                    "Properties": record["NodeAProperties"] if node_a else None,
                    "Values": node_a_props
                },
                "NodeB": {
                    "ID": node_b.element_id if node_b else None,
                    "Properties": record["NodeBProperties"] if node_b else None,
                    "Values": node_b_props
                },
                "Relationship": {
                    "Type": relationship_type if relationship_type else None,
                    "Properties": relationship_props if relationship_type else None,
                    "Values": relationship_values if relationship_type else None
                },
                "MatchedKeywords": matched_keywords,
                "MatchPercentage": match_percentage
            })

        # Sort matches by match percentage in descending order
        sorted_matches = sorted(matches, key=lambda x: x["MatchPercentage"], reverse=True)
        return sorted_matches[:10]  # Return top 10 matches


In [ ]:
def create_keywords(query):
    anchor_node_query = f"""
From the following user query:

'{query}'

Extract the most important **semantic keywords or named entities** that represent meaningful subjects, objects, or actions. 

- Remove stopwords (like "who", "is", "the", etc.)
- Return **nouns**, **proper nouns**, and **verbs** only if they carry meaning (e.g., 'record', 'Elvis', 'song')
- Normalize to singular form and lowercase
- Do NOT include generic pronouns (like "you", "i", "it")
- DO NOT explain or add extra text

Respond ONLY with a Python list of keywords, like: ['record', 'can't','help','fall','love', 'with']
"""

    llm_content = ask_openai(anchor_node_query)
    anchor_nodes = []
    # Keep trying to extract keywords until anchor_nodes is not empty
    while not anchor_nodes:
        llm_content = ask_openai(anchor_node_query)
        pattern = r"\[.*?\]"
        try:
            anchor_nodes = eval(re.findall(pattern, llm_content)[0])
        except:
            anchor_nodes = []  # Keep it empty if extraction fails

        # Print status for each attempt
        if not anchor_nodes:
            print("No keywords extracted, retrying...")

    print(f"Extracted Keywords: {anchor_nodes}")
    matched_results=[]

    # Example Usage
    try:
        # List of keywords extracted
        #keywords = ['record', 'i', 'Can"t', 'Help', 'Fall', 'Love', 'You']
        #keywords = ["apple","iphone","out"]
        keywords = anchor_nodes
        
        # Replace "neo4j" with your database name
        database_name = "neo4j"
        
        # Perform the match
        matched_results = match_keywords_with_relevance(driver, keywords, database_name)
        
        # Display top 10 matches
        print(f"Top 10 Matches for Keywords: {keywords}")
        for i, result in enumerate(matched_results, 1):
            print(f"Rank {i}:")
            print(f"Match Percentage: {result['MatchPercentage']:.2f}%")
            print(f"Matched Keywords: {result['MatchedKeywords']}")
            print(f"Node A (ID: {result['NodeA']['ID']})")
            print(f" - Properties: {result['NodeA']['Properties']}")
            print(f" - Values: {result['NodeA']['Values']}")
            if result['Relationship']['Type']:
                print(f"Relationship ({result['Relationship']['Type']})")
                print(f" - Properties: {result['Relationship']['Properties']}")
                print(f" - Values: {result['Relationship']['Values']}")
            if result['NodeB']['ID']:
                print(f"Node B (ID: {result['NodeB']['ID']})")
                print(f" - Properties: {result['NodeB']['Properties']}")
                print(f" - Values: {result['NodeB']['Values']}")
            print("-" * 40)
    finally:
        print("Done!")
        
    # Extract all documentIds from top match (if exists)
    top_document_ids = set()
    if matched_results:
        top_result = matched_results[0]

        def extract_doc_ids(entity):
            if not entity or not entity.get("Properties") or not entity.get("Values"):
                return []
            return [v for k, v in zip(entity["Properties"], entity["Values"]) if k == "documentId"]

        top_document_ids.update(extract_doc_ids(top_result.get("NodeA", {})))
        top_document_ids.update(extract_doc_ids(top_result.get("Relationship", {})))
        top_document_ids.update(extract_doc_ids(top_result.get("NodeB", {})))

    print(f"Top Document IDs: {top_document_ids}")
    return matched_results, top_document_ids



In [ ]:
# def create_keywords(query):
#     anchor_node_query = f"""Extract the keywords from the given query: '{query}'. Extract keywords in their singular form. Return the answer as a list of keywords. Do not state any other words or explaination in your response. If no keywords present, return only empty list."""
#     llm_content = ollama.invoke(anchor_node_query)
#     anchor_nodes = []
#     # Keep trying to extract keywords until anchor_nodes is not empty
#     while not anchor_nodes:
#         llm_content = ollama.invoke(anchor_node_query)
#         pattern = r"\[.*?\]"
#         try:
#             anchor_nodes = eval(re.findall(pattern, llm_content)[0])
#         except:
#             anchor_nodes = []  # Keep it empty if extraction fails

#         # Print status for each attempt
#         if not anchor_nodes:
#             print("No keywords extracted, retrying...")

#     print(f"Extracted Keywords: {anchor_nodes}")
#     matched_results=[]

#     # Example Usage
#     try:
#         # List of keywords extracted
#         #keywords = ["chicago", "fire","season","4", "released"]  # Replace with your extracted keywords
#         #keywords = ["apple","iphone","out"]
#         keywords = anchor_nodes
        
#         # Replace "neo4j" with your database name
#         database_name = "neo4j"
        
#         # Perform the match
#         matched_results = match_keywords_with_relevance(driver, keywords, database_name)
        
#         # Display top 10 matches
#         print(f"Top 10 Matches for Keywords: {keywords}")
#         for i, result in enumerate(matched_results, 1):
#             print(f"Rank {i}:")
#             print(f"Match Percentage: {result['MatchPercentage']:.2f}%")
#             print(f"Matched Keywords: {result['MatchedKeywords']}")
#             print(f"Node A (ID: {result['NodeA']['ID']})")
#             print(f" - Properties: {result['NodeA']['Properties']}")
#             print(f" - Values: {result['NodeA']['Values']}")
#             if result['Relationship']['Type']:
#                 print(f"Relationship ({result['Relationship']['Type']})")
#                 print(f" - Properties: {result['Relationship']['Properties']}")
#                 print(f" - Values: {result['Relationship']['Values']}")
#             if result['NodeB']['ID']:
#                 print(f"Node B (ID: {result['NodeB']['ID']})")
#                 print(f" - Properties: {result['NodeB']['Properties']}")
#                 print(f" - Values: {result['NodeB']['Values']}")
#             print("-" * 40)
#     finally:
#         print("Done!")
        
#     return matched_results
        

In [ ]:
query="What was XYZ Comapny involved in?"
# Example Usage
try:

    # Query the LLM with the extracted context
    matched_results, top_doc_id = create_keywords(query)
    print(f"*********")
    print(matched_results)
    print(top_doc_id)
    answer = query_llm_with_context(llm, query, matched_results)
    print(f"*********\n Answer: {answer}")
finally:
    print("Done!!")

Extracted Keywords: ['xyz', 'company', 'involved']
Top 10 Matches for Keywords: ['xyz', 'company', 'involved']
Rank 1:
Match Percentage: 33.33%
Matched Keywords: ['company']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2667)
 - Properties: ['id', 'name']
 - Values: ['p_lorillard_company', 'p lorillard and company']
Relationship (ADVERTISED_IN)
 - Properties: ['year']
 - Values: ['1789']
Node B (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2669)
 - Properties: ['id', 'name']
 - Values: ['new_york_daily', 'new york daily']
----------------------------------------
Rank 2:
Match Percentage: 33.33%
Matched Keywords: ['company']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2669)
 - Properties: ['id', 'name']
 - Values: ['new_york_daily', 'new york daily']
Relationship (ADVERTISED_IN)
 - Properties: ['year']
 - Values: ['1789']
Node B (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2667)
 - Properties: ['id', 'name']
 - Values: ['p_lorillard_company', 'p lorillard and company']
--------

In [ ]:
query="recorded can't help falling in love with you"
# Example Usage
try:

    # Query the LLM with the extracted context
    matched_results, top_doc_id = create_keywords(query)
    print(f"*********")
    answer = query_llm_with_context(llm, query, matched_results)
    print(f"*********\n Answer: {answer}")
finally:
    print("Done!!")

Extracted Keywords: ['record', 'help', 'fall', 'love']
Top 10 Matches for Keywords: ['record', 'help', 'fall', 'love']
Rank 1:
Match Percentage: 75.00%
Matched Keywords: ['help', 'fall', 'love']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2559)
 - Properties: ['id', 'name', 'documentId']
 - Values: ['royal_philharmonic_orchestra', 'royal philharmonic orchestra', 'merged_doc11']
Relationship (PERFORMED_WITH)
 - Properties: ['year', 'documentId', 'context']
 - Values: ['2015', 'merged_doc11', 'If I Can Dream album']
Node B (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:3745)
 - Properties: ['id', 'name', 'documentId']
 - Values: ['cant_help_falling_in_love', "can't help falling in love", 'merged_doc11']
----------------------------------------
Rank 2:
Match Percentage: 75.00%
Matched Keywords: ['help', 'fall', 'love']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:3745)
 - Properties: ['id', 'name', 'documentId']
 - Values: ['cant_help_falling_in_love', "can't help falling in lov

In [ ]:
query="what was the name of atom bomb dropped by usa on hiroshima"
# Example Usage
try:

    # Query the LLM with the extracted context
    matched_results, top_doc_id = create_keywords(query)
    print(f"*********")
    answer = query_llm_with_context(llm, query, matched_results)
    print(f"*********\n Answer: {answer}")
finally:
    print("Done!!")

Extracted Keywords: ['atom', 'bomb', 'usa', 'drop', 'hiroshima']
Top 10 Matches for Keywords: ['atom', 'bomb', 'usa', 'drop', 'hiroshima']
Rank 1:
Match Percentage: 60.00%
Matched Keywords: ['atom', 'bomb', 'hiroshima']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2705)
 - Properties: ['id', 'name']
 - Values: ['atomic_bombs', 'atomic bombs']
Relationship (USED_IN)
 - Properties: []
 - Values: []
Node B (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2703)
 - Properties: ['id', 'name', 'date']
 - Values: ['hiroshima_bombing', 'hiroshima bombing', '1945-08-06']
----------------------------------------
Rank 2:
Match Percentage: 60.00%
Matched Keywords: ['atom', 'bomb', 'hiroshima']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2703)
 - Properties: ['id', 'name', 'date']
 - Values: ['hiroshima_bombing', 'hiroshima bombing', '1945-08-06']
Relationship (USED_IN)
 - Properties: []
 - Values: []
Node B (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2705)
 - Properties: ['id', 'name']
 - V

In [ ]:
query="how many episodes are in chicago fire season 4"
# Example Usage
try:

    # Query the LLM with the extracted context
    matched_results, top_doc_id = create_keywords(query)
    print(f"*********")
    print(matched_results)
    print(f"!!!!!")
    answer = query_llm_with_context(llm, query, matched_results)
    print(f"*********\n Answer: {answer}")
finally:
    print("Done!!")

Extracted Keywords: ['episode', 'chicago fire', 'season 4']
Top 10 Matches for Keywords: ['episode', 'chicago fire', 'season 4']
Rank 1:
Match Percentage: 66.67%
Matched Keywords: ['chicago fire', 'season 4']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2524)
 - Properties: ['id', 'name', 'premiereDate', 'endDate', 'episodeCount']
 - Values: ['chicago_fire_season_4', 'chicago fire season 4', '2015-10-13', '2016-05-17', 23]
Relationship (FEATURES)
 - Properties: ['role']
 - Values: ['lieutenant']
Node B (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2531)
 - Properties: ['id', 'name', 'role']
 - Values: ['kelly_severide', 'kelly severide', 'lieutenant']
----------------------------------------
Rank 2:
Match Percentage: 66.67%
Matched Keywords: ['chicago fire', 'season 4']
Node A (ID: 4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2531)
 - Properties: ['id', 'name', 'role']
 - Values: ['kelly_severide', 'kelly severide', 'lieutenant']
Relationship (FEATURES)
 - Properties: ['role']
 - Values:

In [ ]:
def test_neo4j_connection(driver, database="neo4j"):
    with driver.session(database=database) as session:
        result = session.run("MATCH (n) RETURN n LIMIT 5")
        for record in result:
            print(record)

test_neo4j_connection(driver)

<Record n=<Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2515' labels=frozenset({'concept'}) properties={'name': 'minority interest', 'id': 'minority_interest'}>>
<Record n=<Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2516' labels=frozenset({'organization'}) properties={'name': 'parent corporation', 'id': 'parent_corporation'}>>
<Record n=<Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2517' labels=frozenset({'organization'}) properties={'name': 'subsidiary corporation', 'id': 'subsidiary_corporation'}>>
<Record n=<Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2518' labels=frozenset({'person'}) properties={'role': 'investor', 'id': 'investor'}>>
<Record n=<Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2519' labels=frozenset({'concept'}) properties={'name': 'associate company', 'id': 'associate_company'}>>


In [ ]:
def get_graph_overview(driver, database="neo4j"):
    with driver.session(database=database) as session:
        node_labels = session.run("CALL db.labels()").value()
        rel_types = session.run("CALL db.relationshipTypes()").value()
        print(f"Node Labels: {node_labels}")
        print(f"Relationship Types: {rel_types}")

get_graph_overview(driver)


Node Labels: ['song', 'person', 'television_series', 'event', 'location', 'organization', 'agreement', 'law', 'country', 'legislation', 'document', 'character', 'work', 'album', 'concept', 'brand', 'organism', 'campaign', 'infrastructure', 'publication', 'war', 'process', 'product', 'case', 'title', 'guideline', 'other', 'appliance', 'amendment']
Relationship Types: ['RECORDED_BY', 'WRITTEN_BY', 'COMPOSED_BY', 'FEATURED_IN', 'PRODUCED_BY', 'PRODUCED', 'PART_OF', 'DIRECTED_BY', 'RELATED_TO', 'HAD_AFFAIR_WITH', 'MOTHER_OF', 'VISITED', 'RULED_BY', 'CAPITAL_OF', 'DIRECTED', 'RELEASED_BY', 'PERFORMED_BY', 'ACTED_IN', 'LEADS', 'MENTIONED_IN', 'ILLUSTRATED_BY', 'COMPOSED', 'OWNS', 'BROADCASTED_BY', 'INCLUDED_IN', 'MEMBER_OF', 'BORN_IN', 'PARENT_OF', 'WORKED_AT', 'CRITICIZED', 'TARGETED', 'INTRODUCED', 'WROTE', 'LOCATED_IN', 'INCLUDES', 'GIVEN_TO', 'ADMINISTERS', 'CONTRASTS_WITH', 'USES', 'CREATED_BY', 'BASED_ON', 'UTILIZES', 'CONTRIBUTED_TO', 'PROPOSED', 'CROSSES', 'PARTNERED_WITH', 'NAMED_AF

In [ ]:
def debug_match_keywords(driver, keywords, database="neo4j"):
    query = """
    OPTIONAL MATCH (n)-[r]-(m)
    WHERE ANY(keyword IN $keywords WHERE 
        ANY(prop IN keys(n) WHERE 
            n[prop] IS NOT NULL AND toLower(toString(n[prop])) CONTAINS toLower(keyword)
        )
    )
    RETURN n, r, m
    LIMIT 10
    """
    with driver.session(database=database) as session:
        result = session.run(query, keywords=keywords)
        for record in result:
            print("Node A:", record["n"])
            print("Relationship:", record["r"])
            print("Node B:", record["m"])
            print("-" * 30)

debug_match_keywords(driver, ["Chicago Fire", "season", "episode"])


Node A: <Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2524' labels=frozenset({'television_series'}) properties={'premiereDate': '2015-10-13', 'endDate': '2016-05-17', 'episodeCount': 23, 'name': 'chicago fire season 4', 'id': 'chicago_fire_season_4'}>
Relationship: <Relationship element_id='5:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:1153305234164943324' nodes=(<Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2524' labels=frozenset({'television_series'}) properties={'premiereDate': '2015-10-13', 'endDate': '2016-05-17', 'episodeCount': 23, 'name': 'chicago fire season 4', 'id': 'chicago_fire_season_4'}>, <Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2531' labels=frozenset({'person'}) properties={'role': 'lieutenant', 'name': 'kelly severide', 'id': 'kelly_severide'}>) type='FEATURES' properties={'role': 'lieutenant'}>
Node B: <Node element_id='4:d0b1c407-34c5-4596-88bc-641d5b2c8ec2:2531' labels=frozenset({'person'}) properties={'role': 'lieutenant', 'name': 'ke